In [ ]:
import paho.mqtt
import paho.mqtt.client as mqtt_client
import os
from pathlib import Path
import configparser
import queue
import threading

import logging
import random
from logging.handlers import TimedRotatingFileHandler
import sys

ROOT_DIR = os.path.dirname(os.path.abspath(__vsc_ipynb_file__))
CONFIG = Path(ROOT_DIR) / 'hubmqttsrv.ini'

class Mqtt():
    
    
    def __init__(self):
        self.data_queue = queue.Queue()
        # Create threads
        self.threads = []

        self.host = None
        self.port = None
        self.recvmessage = None
        self.onmessage_topic = None

        self.strtpnt = None
        self.dstn = None

        self.PUB_INIT_TOPIC = None
        self.PUB_EDGENODE_TOPIC = None
        self.PUB_CTRLCENTER_TOPIC = None

        self.client_list = dict()

        self.subscribes = None


        self.mysqlctrl = None
    
        client_id = f'publish-{random.randint(0, 1000)}'
        # Old
        #self.client = mqtt_client.Client()

        #self.client = mqtt_client.Client(mqtt_client.CallbackAPIVersion.VERSION1,client_id)

        try:
            if paho.mqtt.__version__[0] > '1':
                self.client = mqtt_client.Client(mqtt_client.CallbackAPIVersion.VERSION1)
            else:
                self.client = mqtt_client.Client()

            #self.client = mqtt_client.Client(mqtt_client.CallbackAPIVersion.VERSION1)
        except Exception as err:
            print("앗 mqtt_client.Client(mqttClient.CallbackAPIVersion.VERSION1,  방식은 문제가 있네")
            print("%s. Reconnect failed. Retrying...", err)
            self.client = mqtt_client.Client()

        return
    
    def __del__(self):
        self.mysqlctrl.close()
        
        for thread in self.threads:
            thread.join()
            
    def on_connect(self, client, userdata, flags, rc):
        if rc == 0:
            logging.info("connected OK")
        else:
            logging.info("Bad connection Returned code=",rc)
    
    def setLogger(self, logger):
        self.logger = logger
        
    def ready(self, _host, _port,_pub_init_topic,_pub_edgenode_topic,_pub_ctrlcenter_topic,_dbinfo):
        self.host = _host
        self.port = _port
        self.PUB_INIT_TOPIC = _pub_init_topic
        self.PUB_EDGENODE_TOPIC = _pub_edgenode_topic
        self.PUB_CTRLCENTER_TOPIC = _pub_ctrlcenter_topic

        

    def start(self, subscribes, retry=False):
        
        if retry is False:
            self.subscribes = subscribes
            self.client.connect(self.host, self.port, keepalive=60)
        self.client.on_connect = self.on_connect
        self.client.on_connect = self.on_connect
        
        for topic in self.subscribes:
            topic = "".join(topic.split())
            logging.info("구독토픽 추가 [{0}]".format(topic))
            self.client.subscribe(topic,0)
            
            
        self.client.loop_forever()
            
def create_rotating_log(path, _config):
    _logger_level = _config['LOG_LEVEL']
    _logger_when = _config['LOG_WHEN']
    _logger_interval = _config['LOG_INTERVAL']
    _logger_backupcount = _config['LOG_BACKUPCOUNT']

    set_level = logging.INFO

    if _logger_level == "INFO":
        set_level = logging.INFO
    elif _logger_level == "DEBUG":
        set_level = logging.DEBUG
    elif _logger_level == "NOTICE":
        set_level = logging.NOTICE
    elif _logger_level == "ERR":
        set_level = logging.ERR
        
    """
    Creates a rotating log
    """
    logging.basicConfig(level=set_level, format='%(asctime)s - %(levelname)s - [%(filename)s:%(lineno)d] %(message)s')
    logger = logging.getLogger("")
    logger.setLevel(set_level)

    print("_logger_level: {0}, _logger_backupcount:{1}, _logger_when: {2}".format(_logger_level, _logger_backupcount, _logger_when ));
    
    # add a rotating handler
    """ 
    handler = RotatingFileHandler(path, maxBytes=100000000,
                                  backupCount=5)
    """
    handler = TimedRotatingFileHandler(path,
                                       when=_logger_when,
                                       interval=int(_logger_interval),
                                       backupCount=int(_logger_backupcount),
                                       encoding='utf-8')
    handler.suffix = "-%Y%m%d_%H-%M-%S"
                                          
    logger.addHandler(handler)

    formatter = logging.Formatter('%(asctime)s - %(levelname)s - [%(filename)s:%(lineno)d] %(message)s' )        
    handler.setFormatter(formatter) # 핸들러에 로깅 포맷 할당    

    return logger        



if __name__ == "__main__":
    _config = configparser.ConfigParser()
    _config.read(CONFIG,encoding='utf-8')
    
    _mqhost = _config['MQTT']['BROKER_HOST']
    _mqport = int(_config['MQTT']['BROKER_PORT'])
    _subscribes = _config['MQTT']['TOPIC_SUBS']
    print(_subscribes)
    subscribes = _subscribes.split(',')
    
    _pub_init_topic = _config['MQTT']['PUB_INIT_TOPIC']
    _pub_edgenode_topic = _config['MQTT']['PUB_EDGENODE_TOPIC']
    _pub_ctrlcenter_topic = _config['MQTT']['PUB_CTRLCENTER_TOPIC']
    
    _logger_level = _config['LOGGER']['LOG_LEVEL']
    _logger_when = _config['LOGGER']['LOG_WHEN']
    _logger_interval = _config['LOGGER']['LOG_INTERVAL']
    _logger_backupcount = _config['LOGGER']['LOG_BACKUPCOUNT']
    
    full_path = os.path.join(ROOT_DIR,"logs","hubmqttsrv.log")\
    
    logger = create_rotating_log(full_path,_config['LOGGER'])
    logging.info("logger path {0}".format(full_path))
    
    mqtt = Mqtt()
    
    mqtt.ready(_mqhost,_mqport,_pub_init_topic,_pub_edgenode_topic,_pub_ctrlcenter_topic,_config['DB'])
    try:
        mqtt.start(subscribes)
    except:
        logger.exception("MQTT Connect Fail...")
        sys.exit()



2024-10-16 11:18:54,273 - INFO - [538773380.py:173] logger path d:\python 관련\PythonDataWorkspace\각종파이썬활용\for문 활용\logs\hubmqttsrv.log
2024-10-16 11:18:54,285 - INFO - [538773380.py:98] 구독토픽 추가 [edgeplatform/init]
2024-10-16 11:18:54,287 - INFO - [538773380.py:98] 구독토픽 추가 [edgeplatform/node/#]
2024-10-16 11:18:54,289 - INFO - [538773380.py:98] 구독토픽 추가 [edgeplatform/ctrlsub/#]
2024-10-16 11:18:54,294 - INFO - [538773380.py:72] connected OK


edgeplatform/init, edgeplatform/node/#, edgeplatform/ctrlsub/#
_logger_level: INFO, _logger_backupcount:30, _logger_when: D


In [9]:
import configparser
import os

ROOT_DIR = os.path.dirname(os.path.abspath(__vsc_ipynb_file__))
CONFIG = Path(ROOT_DIR) / 'hubmqttsrv.ini'

_config = configparser.ConfigParser()
_config.read(CONFIG,encoding='utf-8')

_subscribes = _config['MQTT']['TOPIC_SUBS']
subscribes = _subscribes.split(',')
print(subscribes)

['edgeplatform/init', ' edgeplatform/node/#', ' edgeplatform/ctrlsub/#']
